In [1]:
print("Utviklingsmiljøet er klart!")

Utviklingsmiljøet er klart!


# Beskrivelse av koden for innhenting av værdata

## Introduksjon
Denne koden henter værdata fra Frost API og bruker OpenCage API for å finne koordinatene til en spesifisert sted i Norge. Den benytter funksjoner fra egendefinerte klasser for å håndtere API-nøkler og hente værdata. Dataene prosesseres og lagres i separate CSV-filer for videre analyse.

---

## Funksjoner og klasser brukt i koden
Klassene er lagret under src mappen

### **APIKeyManager**
- **Hensikt**: Håndterer API-nøkler for ulike tjenester.
- **Funksjon brukt**:  
  - `get_api_key(service_name)`: Henter API-nøkkelen for en spesifisert tjeneste (f.eks. Frost eller OpenCage).

### **WeatherDataFetcher**
- **Hensikt**: Håndterer kommunikasjon med Frost API for å hente værdata og værstasjoner.
- **Funksjoner brukt**:  
  - `fetch_stations()`: Henter tilgjengelige værstasjoner fra Frost API.  
  - `fetch_data_by_element(stations, elements, start_date, end_date)`: Henter værdata for spesifikke elementer (f.eks. temperatur, nedbør) fra de nærmeste værstasjonene.  
  - `process_data(data)`: Prosesserer rådata fra API til en Pandas DataFrame for enklere håndtering.

---

## Viktige steg i koden

1. **Initialisering av API-nøkler**:  
   - `APIKeyManager` brukes til å hente API-nøkler for Frost og OpenCage.

2. **Geokoding av lokasjon**:  
   - Bruker OpenCage API til å finne koordinatene for en spesifisert lokasjon.

3. **Henting av værstasjoner**:  
   - `fetch_stations()` henter tilgjengelige værstasjoner fra Frost API.  
   - Avstanden mellom lokasjonen og værstasjonene beregnes ved hjelp av `geopy.distance`.

4. **Henting av værdata**:  
   - `fetch_data_by_element()` henter værdata for spesifikke elementer (f.eks. temperatur, nedbør, vindhastighet, luftfuktighet) for en gitt tidsperiode.

5. **Prosessering av data**:  
   - `process_data()` konverterer rådata til en Pandas DataFrame.  
   - Dataene filtreres basert på elementer og lagres i separate CSV-filer:
     - Temperatur (`temperature_data.csv`)
     - Nedbør (`precipitation_data.csv`)
     - Vindhastighet (`wind_speed_data.csv`)
     - Relativ luftfuktighet (`humidity_data.csv`)

---

## Brukte bibliotek

- **`os` og `sys`**: For å håndtere filstier og importere egendefinerte moduler.
- **`datetime` og `timedelta`**: For å håndtere datoer og tidsperioder.
- **`opencage.geocoder`**: For geokoding av lokasjoner.
- **`geopy.distance`**: For å beregne avstander mellom koordinater.
- **`pandas`**: For å prosessere og lagre data i tabellformat.

---

## Oppsummering
Koden bruker funksjoner fra egendefinerte klasser (`APIKeyManager` og `WeatherDataFetcher`) for å hente og prosessere værdata. Den kombinerer data fra Frost API og OpenCage API for å gi brukeren værdata basert på en spesifisert lokasjon. Resultatet lagres i separate CSV-filer, klare for videre analyse.

In [3]:
# Importer nødvendige biblioteker
import os
import sys
from datetime import datetime, timedelta
from opencage.geocoder import OpenCageGeocode

# Legg til sti for egendefinerte moduler
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

# Importer APIKeyManager og WeatherDataFetcher fra modulen
from api_key_manager_class import APIKeyManager
from var_data_henting_class import WeatherDataFetcher

# Initialiser APIKeyManager
api_key_manager = APIKeyManager()

# Hent Frost API-nøkkelen
client_id = api_key_manager.get_api_key("Frost")

# Hent OpenCage API-nøkkelen
api_key = api_key_manager.get_api_key("Opencage")

# Sett opp OpenCageGeocode med API-nøkkelen
geolocator = OpenCageGeocode(api_key)

# Henter inn lokasjon fra brukeren med validering
while True:
    location_input = input("Enter the location (e.g., Bergen, Oslo, Trondheim, Stavanger, Tromsø): ")
    location = geolocator.geocode(location_input)
    if location:
        location_coords = (location[0]['geometry']['lat'], location[0]['geometry']['lng'])
        print(f"Coordinates for {location_input}: {location_coords}")
        break
    else:
        print("Invalid location. Please try again.")

# Initialiser WeatherDataFetcher med nødvendige parametere
observations_endpoint = 'https://frost.met.no/observations/v0.jsonld'
stations_endpoint = 'https://frost.met.no/sources/v0.jsonld'
weather_fetcher = WeatherDataFetcher(client_id, observations_endpoint, stations_endpoint)

# Hent værstasjoner
weather_stations_coords = weather_fetcher.fetch_stations()

# Regn ut avstanden til værstasjonene fra lokasjonen
from geopy.distance import geodesic
distances = {station: geodesic(location_coords, coords).km for station, coords in weather_stations_coords.items()}

# Sorter værstasjonene etter avstand til lokasjonen
sorted_stations = sorted(distances, key=distances.get)

# Hent ut dagens dato og datoen x dager tilbake
now = datetime.now()
days = 365 * 7
offset = now - timedelta(days)
current_date = now.strftime("%Y-%m-%d")
historic_date = offset.strftime("%Y-%m-%d")

# Elementene vi ønsker å hente data for
elements = [
    'mean(air_temperature P1D)', 
    'sum(precipitation_amount P1D)', 
    'mean(wind_speed P1D)',
    'mean(relative_humidity P1D)'
]

# Hent data for lokasjonen
data = weather_fetcher.fetch_data_by_element(sorted_stations, elements, historic_date, current_date)

# Prosesser dataene til en Pandas DataFrame
df = weather_fetcher.process_data(data)

# Lagrer dataene i CSV-filer (overskriver eksisterende fil)

# Filtrerer og lagrer data for temperatur
temperature_data = df[df['elementId'] == 'mean(air_temperature P1D)']
temperature_data.to_csv("../data/temperature_data.csv", index=False)

# Filtrerer og lagrer data for nedbør
precipitation_data = df[df['elementId'] == 'sum(precipitation_amount P1D)']
precipitation_data.to_csv("../data/precipitation_data.csv", index=False)

# Filtrerer og lagrer data for vindhastighet
wind_speed_data = df[df['elementId'] == 'mean(wind_speed P1D)']
wind_speed_data.to_csv("../data/wind_speed_data.csv", index=False)

# Filtrerer og lagrer data for relativ luftfuktighet
humidity_data = df[df['elementId'] == 'mean(relative_humidity P1D)']
humidity_data.to_csv("../data/humidity_data.csv", index=False)

Coordinates for Tromsø: (69.6516345, 18.9558585)
Henter data for element: mean(air_temperature P1D)
Data retrieved for mean(air_temperature P1D) from station SN90450!
Henter data for element: sum(precipitation_amount P1D)
Data retrieved for sum(precipitation_amount P1D) from station SN90450!
Henter data for element: mean(wind_speed P1D)
Data retrieved for mean(wind_speed P1D) from station SN90450!
Henter data for element: mean(relative_humidity P1D)
Data retrieved for mean(relative_humidity P1D) from station SN90450!
